# 03-01 : Local Deploy

Run MLRun locally in docker and run a function from hub.

## 1. Perpare Environment

In [1]:
import os

In [2]:
# delete the mlrun-data directory
!rm -rf ~/mlrun-data

In [3]:
# delete docker volumes
!docker volume rm 02_docker_instance_nuclio-platform-config
!docker volume rm 03_docker_local_nuclio-platform-config
!docker volume rm local-mrlrun_nuclio-platform-config
!docker volume rm nuclio-local-storage
!docker volume rm nuclio-nuclio-quick-tutorial-jovyan-serving

Error response from daemon: get 02_docker_instance_nuclio-platform-config: no such volume
Error response from daemon: get 03_docker_local_nuclio-platform-config: no such volume
Error response from daemon: get local-mrlrun_nuclio-platform-config: no such volume
Error response from daemon: get nuclio-local-storage: no such volume
Error response from daemon: get nuclio-nuclio-quick-tutorial-jovyan-serving: no such volume


In [4]:
# get the server (local) IP address to a local variable
server_ip = !(hostname -I | awk '{print $1}')
server_ip = server_ip[0]
print(f"Server IP: {server_ip}")


Server IP: 192.168.1.184


In [5]:
# create the mlrun environment file
with open("mlrun.env", "w") as f:
    f.write(f"MLRUN_DBPATH=http://{server_ip}:8080\n")

In [6]:
# create the config file
with open("config.env", "w") as f:
    f.write(f"HOST_IP={server_ip}\n")
    f.write(f"SHARED_DIR=~/mlrun-data\n")

## 2. Deploy the MLRun

In [7]:
# deploy the mlrun server
!docker-compose -f compose.yaml --env-file config.env up -d

WARN[0000] a network with name mlrun exists but was not created for project "03_docker_local".
Set `external: true` to use an existing network 
[+] Running 2/0
 ✔ Network 03_docker_local_default                  Created                0.1s 
 ✔ Volume "03_docker_local_nuclio-platform-config"  Created                0.0s 
 ⠋ Container 03_docker_local-mlrun-api-1            Creating               0.0s 
 ⠋ Container 03_docker_local-mlrun-ui-1             Creating               0.0s 
 ⠋ Container 03_docker_local-init_nuclio-1          Creating               0.0s 
[+] Running 3/5
 ✔ Network 03_docker_local_default                  Created                0.1s 
 ✔ Volume "03_docker_local_nuclio-platform-config"  Created                0.0s 
 ⠙ Container 03_docker_local-mlrun-api-1            Starting               0.1s 
 ⠙ Container 03_docker_local-mlrun-ui-1             Starting               0.1s 
 ⠙ Container 03_docker_local-init_nuclio-1          Starting               0.1s 
 ✔ Container 0

In [8]:
!docker ps -a

CONTAINER ID   IMAGE                                   COMMAND                  CREATED                  STATUS                                     PORTS                                                                                                                             NAMES
bdf41049b57e   gcr.io/iguazio/alpine:3.20              "/bin/sh -c '/bin/sl…"   Less than a second ago   Created                                                                                                                                                                      nuclio-local-storage-reader
209105c4fc1d   quay.io/nuclio/dashboard:stable-amd64   "/docker-entrypoint.…"   1 second ago             Up Less than a second (health: starting)   80/tcp, 0.0.0.0:8070->8070/tcp, :::8070->8070/tcp                                                                                 03_docker_local-nuclio-1
b8c3292a6246   mlrun/mlrun-ui:1.7.2                    "/docker-entrypoint.…"   1 second ago             Up

In [9]:
# remove deployed containers
#!docker-compose -f compose.yaml --env-file config.env down